In [1]:
import numpy as np
import matplotlib.pyplot as plt
#from scripts.utils import info_map
#from scripts.arena import Arena, Aperture, Mouse, visualize_arena
from tqdm import tqdm
import os
from scipy.interpolate import interp1d
import torch
import torch.optim as optim

BASE_PATH = "/home/rdkeller/active_sensing/act_sense/data/"

In [2]:
data = np.load("/Users/reecekeller/Documents/Xaq/act_sense/act_sense/data/info_matrix_10.0.npy")
data.shape

(60, 50)

In [3]:
def max_information_score(info_matrix, start, end):
    rows, cols = len(info_matrix), len(info_matrix[0])
    start_x, start_y = start
    end_x, end_y = end

    # Initialize DP table
    dp = [[-float('inf')] * cols for _ in range(rows)]
    dp[start_x][start_y] = info_matrix[start_x][start_y]

    # Fill DP table
    for i in range(start_x, rows):
        for j in range(start_y, cols):
            if i == start_x and j == start_y:
                continue
            if i > 0:
                dp[i][j] = max(dp[i][j], dp[i-1][j] + info_matrix[i][j])
            if j > 0:
                dp[i][j] = max(dp[i][j], dp[i][j-1] + info_matrix[i][j])
            if i > 0 and j > 0:
                dp[i][j] = max(dp[i][j], dp[i-1][j-1] + info_matrix[i][j])

    # Backtrack to find the path
    path = []
    x, y = end_x, end_y
    while (x, y) != (start_x, start_y):
        path.append((x, y))
        if x > 0 and dp[x][y] == dp[x-1][y] + info_matrix[x][y]:
            x -= 1
        elif y > 0 and dp[x][y] == dp[x][y-1] + info_matrix[x][y]:
            y -= 1
        else:
            x -= 1
            y -= 1
    path.append((start_x, start_y))
    path.reverse()

    return dp[end_x][end_y], path

# Example usage
info_matrix = data
start = (0, 30)
end = (59, 0)

max_score, optimal_path = max_information_score(info_matrix, start, end)
print("Max Information Score:", max_score)
print("Optimal Path:", optimal_path)

: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_optimal_trajectory(info_matrix, optimal_path):
    # Convert the information matrix to a numpy array for plotting
    info_matrix = np.array(info_matrix)
    
    # Create a colormap plot of the information matrix
    plt.imshow(info_matrix, cmap='viridis', origin='lower')
    plt.colorbar(label='Information Score')
    
    # Extract the x and y coordinates of the optimal path
    x_coords, y_coords = zip(*optimal_path)
    
    # Plot the optimal path as red dots
    plt.plot(y_coords, x_coords, 'ro-', markersize=5, label='Optimal Path')
    
    # Add labels and title
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.title('Optimal Trajectory on Information Matrix')
    plt.legend()
    
    # Show the plot
    plt.show()

# Example usage
info_matrix = data
start = (0, 30)
end = (59, 0)

max_score, optimal_path = max_information_score(info_matrix, start, end)
print("Max Information Score:", max_score)
print("Optimal Path:", optimal_path)

# Plot the optimal trajectory
plot_optimal_trajectory(info_matrix, optimal_path)